<a href="https://colab.research.google.com/github/mavalliani/Semantic-Similarity-of-Sentences/blob/master/Jaccard_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import nltk
import spacy
import os
import string
import gensim

In [0]:
train = pd.read_csv("/content/train_2.csv")
train = train.dropna()
train.head()

ParserError: ignored

In [0]:
def jaccard_similarity(df, col1, col2):
    df["jc_similarity"] = df.apply(lambda x: len(set(x[col1]).intersection(set(x[col2])))/len(set(x[col1]).union(set(x[col2]))), axis=1)
    return df
train = jaccard_similarity(train, "Q1", "Q2")
train.head()

,id,qid1,qid2,question1,question2,is_duplicate,Q1,Q2,jc_similarity
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,"['step', 'step', 'guide', 'invest', 'share', '...","['step', 'step', 'guide', 'invest', 'share', '...",1.000000
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,"['story', 'kohinoor', 'kohinoor', 'diamond']","['would', 'happen', 'indian', 'government', 's...",0.615385
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,"['increase', 'speed', 'internet', 'connection'...","['internet', 'speed', 'increased', 'hacking', ...",0.761905
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,"['mentally', 'lonely', 'solve']","['find', 'remainder', 'math2324math', 'divided...",0.478261
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"['one', 'dissolve', 'water', 'quikly', 'sugar'...","['fish', 'would', 'survive', 'salt', 'water']",0.642857


In [0]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(train[["jc_similarity"]], train["is_duplicate"], random_state=5, test_size=0.2)

In [0]:
# baseline model
print("length of model", len(train))
print("ratio of duplicate",train.is_duplicate.value_counts(normalize=True)[1]*100)
print("ratio of dissimilar", train.is_duplicate.value_counts(normalize=True)[0]*100)

length of model 166509
ratio of duplicate 21.926742698592868
ratio of dissimilar 38.715624981232246


In [0]:
import xgboost as xgb
options = {"model_param":
         {"eta": 0.15, 
         "max_depth": 4,
         "n_estimators":50,
         "sub_sample":0.5,
         "scale_pos_weight":1.75,
         "random_state":1,
         "eval_metric": "logloss",
         "objective": "binary:logistic"},
         "num_round":50,
         "test_frac":0.2}

In [0]:
#training model
xgbmatrix = xgb.DMatrix(train_X, train_y)
xgb_model = xgb.train(options["model_param"], xgbmatrix, options["num_round"], verbose_eval=10)
xgb_valmatrix = xgb.DMatrix(val_X)
y_val_pred = xgb_model.predict(xgb_valmatrix)
y_val_pred #just checking

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


TypeError: ignored

In [0]:
y_val_pred_binary = [1 if y>0.5 else 0 for y in y_val_pred]
from sklearn.metrics import confusion_matrix, classification_report

confusion_matrix(val_y, y_val_pred_binary)

In [0]:
from sklearn.metrics import accuracy_score
accuracy_score(val_y, y_val_pred_binary)